# Workwheet Week 4

In [ ]:
import numpy as np

In [ ]:
W = np.array([[1,1],[1,1]])
e = np.array([0,-1])
w = np.array([1,-2])
b = 0
#x = np.array([[0,0],[0,1],[1,0],[1,1]])
x = np.array([0,0])

In [ ]:
z = max(np.transpose(W).dot(x) + e) + b

## Loading data and libraries

In [1]:
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
X = iris.data
y = iris.target
# N is the sample size - 150
# Di is the number of input features - 4
# Do is the number of output features - 3
# H1 is the number of units in the hidden layer - 10
[N, Di] = X.shape # N objects, Di input dimension
Do = 3 # output dimension
H1 = 10 # dimension of hidden layer
alpha = 0.00001 # set the learning rate for gradient descent
y = np.reshape(y, (N,1)) # convert y to a vector y.shape = (150,1)




   Note that the number of hidden units H1 is set to 10. This can be changed if wanted.

In [5]:
## convert the set of categories in y to a set of one hot variables
from sklearn.preprocessing import OneHotEncoder


In [6]:
onehot_encoder = OneHotEncoder(sparse=False)
y = onehot_encoder.fit_transform(y)

C:\Users\Fabio\Anaconda3\envs\openpose\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


The last step of preparation is to create the set of neural network parameters (weight matrices). We
are creating a two layer network, therefore, weights are needed from the input layer to the hidden layer,
and the hidden layer to the output layer.

In [7]:
np.random.seed(1)

In [8]:
w2 = np.random.randn(Di, H1)
w1 = np.random.randn(H1, Do)

## Loss Function

We choose a simple linear loss function, where we assume that the response ˆy is Normal, and so need
to maximise the likelihood of a Normal distribution with constant variance. This provides the mean
squared error loss and loss gradient.

In [9]:
def linearLoss(y, yhat):
    return(np.square(yhat - y).sum())
def linearLossGrad(y, yhat):
    return(2*(yhat - y))

In [10]:
# set loss function and  Linear function
lossFunction = linearLoss
lossGrad = linearLossGrad

## Unit activation function

In [11]:
def relu(x):
    return(np.maximum(x, 0))
def reluGrad(x):
    if (x < 0):
        return(0)
    return(1)

In [12]:
activation = np.vectorize(relu)
activationGrad = np.vectorize(reluGrad)

In [13]:
activation(np.array([[1,-2],[3,4]]))

array([[1, 0],
       [3, 4]])

In [14]:
activationGrad(np.array([[1,-2],[3,4]]))

array([[1, 0],
       [1, 1]])

## Training the network

Training consists of repeatedly performing:
1. forward propagation to compute the loss, then
2. back propagation to compute the gradient of the loss with respect to the weights, then
3. update the weights using gradient descent

In [15]:
X[0:3]


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

## Forward propagation to compute the estimate of y

In [16]:
h1 = X.dot(w2) # multiply input by weights and sum #  # shape (150,10)
fh1 = activation(h1) # make sure activation function is chosen
yhat = fh1.dot(w1) # output will be shape(150x3)

In [17]:
fh1[0]

array([11.72234014,  0.        ,  0.        ,  0.        ,  9.50872106,
        0.        ,  7.89953289,  0.        ,  1.73176463,  1.65897082])

In [18]:
yhat[0:3]

array([[ -7.48931178,  -3.239756  , -17.06118292],
       [ -6.92492996,  -2.79802819, -15.67824111],
       [ -6.8458251 ,  -2.93272115, -15.63932592]])

In [19]:
loss = lossFunction(y, yhat)

In [20]:
loss

49173.2604865844

In [21]:
for iteration in range(1, 1000):
    
    h1 = X.dot(w2) # multiply input by weights and sum #  # shape (150,10)
    fh1 = activation(h1) # make sure activation function is chosen
    yhat = fh1.dot(w1) # output will be shape(150x3)
    loss = lossFunction(y, yhat)
    
    print('[ iter:', iteration, '] loss:', loss, end='\r', flush=True)
    
    dJdy = lossGrad(y, yhat) # loss gradient
    dydw1 = fh1
    dJdw1 = dydw1.T.dot(dJdy)
    
    ## backpropagate the error compute the gradient with respect to the
    ## second set of weights
    dydh = w1
    dhdg = activationGrad(h1)
    dgdw2 = X
    dJdh = dJdy.dot(dydh.T)
    dJdg = dJdh*dhdg
    dJdw2 = dgdw2.T.dot(dJdg)
    
    #update the weights using gradient descent
    w1 = w1 - alpha*dJdw1
    w2 = w2 - alpha*dJdw2
    
    